In [1]:
import h5py
import numpy as np
import torch
from datetime import date,datetime

In [2]:
# Make sure we have a GPU allocated
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
use_cuda = torch.cuda.is_available()
print(device) # The output should be 'cuda'

cuda


## Splitting

In [3]:
n_split1 = 255000 #spaeter 250000
n_split2 = 255000
n_gesamt = n_split1 + n_split2
batchsize = 256 #baseline 256 #1000

training_percentage = 80 
validation_percentage = 10 
test_percentage = 10 

In [3]:
start = datetime.now()

input_filename =  "/beegfs/desy/user/diefenbs/shower_data/pion_uniform_510k_PunchThroughCut70.hdf5"
f_in = h5py.File(input_filename, 'r') ## 'r'=readable

assert(n_split1+n_split2 <= (f_in['hcal_only']['layers'].shape[0]))

output_filename_split1 = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_orig.hdf5" 
output_filename_split2 = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo.hdf5"
output_filename_split1_validation = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_orig_validation.hdf5" 
output_filename_split2_validation = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_validation.hdf5"
output_filename_split1_test = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_orig_test.hdf5" 
output_filename_split2_test = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_test.hdf5"


f_out = h5py.File(output_filename_split1,'w-')
grp = f_out.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f2', compression="gzip", compression_opts=1)


for index in np.arange(0, int(n_split1*training_percentage*0.01), step=batchsize):
    
    ds1 = f_in['hcal_only']['layers'][index:index+batchsize]
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    ds1E = f_in['hcal_only']['energy'][index:index+batchsize]   
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    ds1L = torch.zeros((batchsize,1)) 
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    

    if index%1000 == 0:
        print(index, ds1E[0],ds1L.shape)

print('pion_orig:')        
print(f_out['hcal_only']['layers'].shape[0])

f_out.close()




f_out = h5py.File(output_filename_split1_validation,'w-')
grp = f_out.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f2', compression="gzip", compression_opts=1)


for index in np.arange(int(n_split1*training_percentage*0.01), int(n_split1*(training_percentage + validation_percentage)*0.01), step=batchsize):    
    
    ds1 = f_in['hcal_only']['layers'][index:index+batchsize]
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    ds1E = f_in['hcal_only']['energy'][index:index+batchsize]   
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    ds1L = torch.zeros((batchsize,1)) 
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    

    if index%1000 == 0:
        print(index, ds1E[0],ds1L.shape)

print('pion_orig_validation:')        
print(f_out['hcal_only']['layers'].shape[0])

f_out.close()




f_out = h5py.File(output_filename_split1_test,'w-')
grp = f_out.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f2', compression="gzip", compression_opts=1)


for index in np.arange(int(n_split1*(training_percentage + validation_percentage)*0.01), n_split1, step=batchsize): 
    
    ds1 = f_in['hcal_only']['layers'][index:index+batchsize]
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    ds1E = f_in['hcal_only']['energy'][index:index+batchsize]   
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    ds1L = torch.zeros((batchsize,1)) 
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    

    if index%1000 == 0:
        print(index, ds1E[0],ds1L.shape)

print('pion_orig_test:')        
print(f_out['hcal_only']['layers'].shape[0])

f_out.close()




f_out = h5py.File(output_filename_split2,'w-')
grp = f_out.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f2', compression="gzip", compression_opts=1)


for index in np.arange(n_split1, n_split1 + int(n_split2*training_percentage*0.01) ,step=batchsize):
    
    ds1 = f_in['hcal_only']['layers'][index:index+batchsize]
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    ds1E = f_in['hcal_only']['energy'][index:index+batchsize]   
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E

    ds1L = torch.ones((batchsize,1))  
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    
    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape)        

print('pion_trafo:')
print(f_out['hcal_only']['layers'].shape[0])

f_out.close()




f_out = h5py.File(output_filename_split2_validation,'w-')
grp = f_out.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f2', compression="gzip", compression_opts=1)


for index in np.arange(n_split1 + int(n_split2*training_percentage*0.01) , n_split1 + int(n_split2*(training_percentage + validation_percentage)*0.01) ,step=batchsize):
    
    ds1 = f_in['hcal_only']['layers'][index:index+batchsize]
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    ds1E = f_in['hcal_only']['energy'][index:index+batchsize]   
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E

    ds1L = torch.ones((batchsize,1))  
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    
    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape)        

print('pion_trafo_validation:')
print(f_out['hcal_only']['layers'].shape[0])      

f_out.close() 




f_out = h5py.File(output_filename_split2_test,'w-')
grp = f_out.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f2', compression="gzip", compression_opts=1)


for index in np.arange(n_split1 + int(n_split2*(training_percentage + validation_percentage)*0.01) , n_split1+n_split2 ,step=batchsize):
    
    ds1 = f_in['hcal_only']['layers'][index:index+batchsize]
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    ds1E = f_in['hcal_only']['energy'][index:index+batchsize]   
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E

    ds1L = torch.ones((batchsize,1))  
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    
    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape)        

print('pion_trafo_test:')
print(f_out['hcal_only']['layers'].shape[0])

f_out.close() 

end = datetime.now()
print(f'total time: {end - start}')

0 [28.150738] torch.Size([256, 1])
32000 [39.779995] torch.Size([256, 1])
64000 [30.451612] torch.Size([256, 1])
96000 [92.544975] torch.Size([256, 1])
128000 [56.029053] torch.Size([256, 1])
160000 [40.94045] torch.Size([256, 1])
192000 [35.8835] torch.Size([256, 1])
pion_orig:
204032
204000 [57.27977] torch.Size([256, 1])
pion_orig_validation:
25600
pion_orig_test:
25600
255000 [21.80484] torch.Size([256, 1])
287000 [56.13993] torch.Size([256, 1])
319000 [28.610886] torch.Size([256, 1])
351000 [59.066257] torch.Size([256, 1])
383000 [12.57898] torch.Size([256, 1])
415000 [20.492687] torch.Size([256, 1])
447000 [32.287186] torch.Size([256, 1])
pion_trafo:
204032
459000 [55.703934] torch.Size([256, 1])
pion_trafo_validation:
25600
pion_trafo_test:
25600


# Define Transformations

In [4]:
def test_trafo_Gauss_Sigma01(x):
    return x*np.random.normal(loc=1.0, scale=0.1, size=x.shape)

def test_trafo_Gauss_Sigma025(x):
    return x*np.random.normal(loc=1.0, scale=0.25, size=x.shape)

def test_trafo_Gauss_Sigma05(x):
    return x*np.random.normal(loc=1.0, scale=0.5, size=x.shape)

def test_trafo_Gauss_Sigma075(x):
    return x*np.random.normal(loc=1.0, scale=0.75, size=x.shape)

def test_trafo_Gauss_Sigma1(x):
    return x*np.random.normal(loc=1.0, scale=1.0, size=x.shape)

## Trafo - 0.1 sigma

In [ ]:
start = datetime.now()

sigma_path = '0,1' #for the outputpath

input_filename = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo.hdf5"
input_filename_validation = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_validation.hdf5"
input_filename_test = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_test.hdf5"

f_in = h5py.File(input_filename,'r')
f_in_len = f_in['hcal_only']['layers'].shape[0]
print(f'f_in_len: {f_in_len}')


output_filename = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/510k all/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma.hdf5"
f_out = h5py.File(output_filename,'w-')


grp = f_out.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len, step=batchsize):  #int(n_split1*training_percentage*0.01)
    
    ds1 = f_in['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma01(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out['hcal_only']['layers'].shape[0])

f_out.close()




f_in_validation = h5py.File(input_filename_validation,'r')
f_in_len_validation = f_in_validation['hcal_only']['layers'].shape[0]
print(f'f_in_len_validation: {f_in_len_validation}')

        
output_filename_validation = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma_validation.hdf5"
f_out_validation = h5py.File(output_filename_validation,'w-')


grp = f_out_validation.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len_validation, step=batchsize):
    
    ds1 = f_in_validation['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma01(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in_validation['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in_validation['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out_validation['hcal_only']['layers'].shape[0])
        
f_out_validation.close()




f_in_test = h5py.File(input_filename_test,'r')
f_in_len_test = f_in_test['hcal_only']['layers'].shape[0]
print(f'f_in_len_test: {f_in_len_test}')


output_filename_test = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma_test.hdf5"
f_out_test = h5py.File(output_filename_test,'w-')


grp = f_out_test.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len_test, step=batchsize):
    
    ds1 = f_in_test['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma01(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in_test['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in_test['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out_test['hcal_only']['layers'].shape[0])
        
f_out_test.close()

end = datetime.now()
print(f'total time: {end - start}')

f_in_len: 204032
0 [21.80484] torch.Size([256, 1]) <class 'torch.Tensor'>
32000 [56.13993] torch.Size([256, 1]) <class 'torch.Tensor'>


## Trafo - 0.25 sigma

In [ ]:
start = datetime.now()

sigma_path = '0,25' #for the outputpath

input_filename = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo.hdf5"
input_filename_validation = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_validation.hdf5"
input_filename_test = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_test.hdf5"

f_in = h5py.File(input_filename,'r')
f_in_len = f_in['hcal_only']['layers'].shape[0]
print(f'f_in_len: {f_in_len}')


output_filename = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/510k all/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma.hdf5"
f_out = h5py.File(output_filename,'w-')


grp = f_out.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len, step=batchsize):  #int(n_split1*training_percentage*0.01)
    
    ds1 = f_in['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma025(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out['hcal_only']['layers'].shape[0])

f_out.close()




f_in_validation = h5py.File(input_filename_validation,'r')
f_in_len_validation = f_in_validation['hcal_only']['layers'].shape[0]
print(f'f_in_len_validation: {f_in_len_validation}')

        
output_filename_validation = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma_validation.hdf5"
f_out_validation = h5py.File(output_filename_validation,'w-')


grp = f_out_validation.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len_validation, step=batchsize):
    
    ds1 = f_in_validation['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma025(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in_validation['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in_validation['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out_validation['hcal_only']['layers'].shape[0])
        
f_out_validation.close()




f_in_test = h5py.File(input_filename_test,'r')
f_in_len_test = f_in_test['hcal_only']['layers'].shape[0]
print(f'f_in_len_test: {f_in_len_test}')


output_filename_test = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma_test.hdf5"
f_out_test = h5py.File(output_filename_test,'w-')


grp = f_out_test.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len_test, step=batchsize):
    
    ds1 = f_in_test['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma025(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in_test['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in_test['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out_test['hcal_only']['layers'].shape[0])
        
f_out_test.close()

end = datetime.now()
print(f'total time: {end - start}')

## Trafo - 0.5 sigma

In [ ]:
start = datetime.now()

sigma_path = '0,5' #for the outputpath

input_filename = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo.hdf5"
input_filename_validation = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_validation.hdf5"
input_filename_test = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_test.hdf5"

f_in = h5py.File(input_filename,'r')
f_in_len = f_in['hcal_only']['layers'].shape[0]
print(f'f_in_len: {f_in_len}')


output_filename = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/510k all/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma.hdf5"
f_out = h5py.File(output_filename,'w-')


grp = f_out.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len, step=batchsize):  #int(n_split1*training_percentage*0.01)
    
    ds1 = f_in['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma05(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out['hcal_only']['layers'].shape[0])

f_out.close()




f_in_validation = h5py.File(input_filename_validation,'r')
f_in_len_validation = f_in_validation['hcal_only']['layers'].shape[0]
print(f'f_in_len_validation: {f_in_len_validation}')

        
output_filename_validation = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma_validation.hdf5"
f_out_validation = h5py.File(output_filename_validation,'w-')


grp = f_out_validation.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len_validation, step=batchsize):
    
    ds1 = f_in_validation['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma05(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in_validation['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in_validation['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out_validation['hcal_only']['layers'].shape[0])
        
f_out_validation.close()




f_in_test = h5py.File(input_filename_test,'r')
f_in_len_test = f_in_test['hcal_only']['layers'].shape[0]
print(f'f_in_len_test: {f_in_len_test}')


output_filename_test = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma_test.hdf5"
f_out_test = h5py.File(output_filename_test,'w-')


grp = f_out_test.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len_test, step=batchsize):
    
    ds1 = f_in_test['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma05(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in_test['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in_test['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out_test['hcal_only']['layers'].shape[0])
        
f_out_test.close()

end = datetime.now()
print(f'total time: {end - start}')

## Trafo - 0.75 sigma

In [ ]:
start = datetime.now()

sigma_path = '0,75' #for the outputpath

input_filename = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo.hdf5"
input_filename_validation = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_validation.hdf5"
input_filename_test = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_test.hdf5"

f_in = h5py.File(input_filename,'r')
f_in_len = f_in['hcal_only']['layers'].shape[0]
print(f'f_in_len: {f_in_len}')


output_filename = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/510k all/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma.hdf5"
f_out = h5py.File(output_filename,'w-')


grp = f_out.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len, step=batchsize):  #int(n_split1*training_percentage*0.01)
    
    ds1 = f_in['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma075(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out['hcal_only']['layers'].shape[0])

f_out.close()




f_in_validation = h5py.File(input_filename_validation,'r')
f_in_len_validation = f_in_validation['hcal_only']['layers'].shape[0]
print(f'f_in_len_validation: {f_in_len_validation}')

        
output_filename_validation = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma_validation.hdf5"
f_out_validation = h5py.File(output_filename_validation,'w-')


grp = f_out_validation.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len_validation, step=batchsize):
    
    ds1 = f_in_validation['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma075(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in_validation['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in_validation['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out_validation['hcal_only']['layers'].shape[0])
        
f_out_validation.close()




f_in_test = h5py.File(input_filename_test,'r')
f_in_len_test = f_in_test['hcal_only']['layers'].shape[0]
print(f'f_in_len_test: {f_in_len_test}')


output_filename_test = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma_test.hdf5"
f_out_test = h5py.File(output_filename_test,'w-')


grp = f_out_test.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len_test, step=batchsize):
    
    ds1 = f_in_test['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma075(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in_test['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in_test['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out_test['hcal_only']['layers'].shape[0])
        
f_out_test.close()

end = datetime.now()
print(f'total time: {end - start}')

## Trafo - 1.0 sigma

In [ ]:
start = datetime.now()

sigma_path = '1,0' #for the outputpath

input_filename = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo.hdf5"
input_filename_validation = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_validation.hdf5"
input_filename_test = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_test.hdf5"

f_in = h5py.File(input_filename,'r')
f_in_len = f_in['hcal_only']['layers'].shape[0]
print(f'f_in_len: {f_in_len}')


output_filename = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/510k all/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma.hdf5"
f_out = h5py.File(output_filename,'w-')


grp = f_out.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len, step=batchsize):  #int(n_split1*training_percentage*0.01)
    
    ds1 = f_in['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma1(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out['hcal_only']['layers'].shape[0])

f_out.close()




f_in_validation = h5py.File(input_filename_validation,'r')
f_in_len_validation = f_in_validation['hcal_only']['layers'].shape[0]
print(f'f_in_len_validation: {f_in_len_validation}')

        
output_filename_validation = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma_validation.hdf5"
f_out_validation = h5py.File(output_filename_validation,'w-')


grp = f_out_validation.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len_validation, step=batchsize):
    
    ds1 = f_in_validation['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma1(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in_validation['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in_validation['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out_validation['hcal_only']['layers'].shape[0])
        
f_out_validation.close()




f_in_test = h5py.File(input_filename_test,'r')
f_in_len_test = f_in_test['hcal_only']['layers'].shape[0]
print(f'f_in_len_test: {f_in_len_test}')


output_filename_test = f"/beegfs/desy/user/schreibj/Hadron_classifier/shower_data/pion_{int(n_gesamt/1000)}k_trafo_{sigma_path}Sigma_test.hdf5"
f_out_test = h5py.File(output_filename_test,'w-')


grp = f_out_test.create_group("hcal_only")
dset = grp.create_dataset('layers', shape=(0, 48, 48, 48), maxshape=(None, 48, 48, 48), 
                          chunks=(100, 48, 48, 48), dtype='f8', compression="gzip", compression_opts=1)
dsetE = grp.create_dataset('energy', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)
dsetL = grp.create_dataset('label', shape=(0, 1), maxshape=(None, 1), chunks=(100, 1), 
                           dtype='f4', compression="gzip", compression_opts=1)


for index in np.arange(0, f_in_len_test, step=batchsize):
    
    ds1 = f_in_test['hcal_only']['layers'][index:index+batchsize]
    
    #######
    
    # Trafo in showers (in ds1) here
    
    ds1 = test_trafo_Gauss_Sigma1(ds1)
    
    #######
    
    dset.resize(dset.shape[0]+ds1.shape[0], axis=0)
    dset[-1*ds1.shape[0]:] = ds1

    
    
    
    ds1E = f_in_test['hcal_only']['energy'][index:index+batchsize]   
    
    #######
    
    # Trafo in energy labels (in ds1E) here
    
    #######
    
    dsetE.resize(dsetE.shape[0]+ds1E.shape[0], axis=0)
    dsetE[-1*ds1E.shape[0]:] = ds1E
    
    
    
    
    ds1L = f_in_test['hcal_only']['label'][index:index+batchsize]
    
    #######
    
    # Trafo in labels here
    
    
    #######
    
    dsetL.resize(dsetL.shape[0]+ds1L.shape[0], axis=0)
    dsetL[-1*ds1L.shape[0]:] = ds1L
    ds1L = torch.from_numpy(ds1L)

    if index%1000 == 0:
        print(index, ds1E[0], ds1L.shape, type(ds1L))

print('pion_trafo_test:')
print(f_out_test['hcal_only']['layers'].shape[0])
        
f_out_test.close()

end = datetime.now()
print(f'total time: {end - start}')